In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pwd

'C:\\Users\\soyma\\Documents\\GitHub\\ERM_2022\\ROP'

### Partidos políticos

In [3]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://aplicaciones007.jne.gob.pe/srop_publico/Consulta/OrganizacionPolitica'
driver.get( url )



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\soyma\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [4]:
# Store all_tables
all_tables = {}

In [5]:
for dpt_idx in range( 1 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    dpt = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[1]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    # We select a different department name
    if dpt_name == "Partido Político" :
        
        # click on department
        department.click()
        
        time.sleep(3)
        
        # Get all elements of province
        prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "--TODOS--" :
                
                # click on province
                province.click()
                
                driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[3]/input').click()
                        
                time.sleep(10)
                
                link = driver.current_url
                
                table_path = driver.find_element_by_xpath( "/html/body/div[1]/div[3]" )
                table_html = table_path.get_attribute( 'innerHTML' )
                table = pd.read_html( table_html )
                        
                table_clean = table[0].iloc[ 0:, [0,1,2]].copy()
                                
                table_clean[ 'department' ] = dpt_name
                table_clean[ 'province' ]   = prov_name
                
                all_tables[ province ] = table_clean

In [6]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )

In [7]:
final_data.to_excel( 'partido_politico.xlsx' , index = False )

In [8]:
driver.quit()

### Movimiento regional

#### - Inscrito

In [9]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://aplicaciones007.jne.gob.pe/srop_publico/Consulta/OrganizacionPolitica'
driver.get( url )



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\soyma\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [10]:
# Store all_tables
all_tables_2 = {}

In [11]:
# we can loop over all departments
# for dpt_idx in range( num_prov_options ):
# but it will take too much time
# We are going to do it over two departments
for dpt_idx in range( 2 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    dpt = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[1]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    # We select a different department name
    if dpt_name == "Movimiento Regional" :
        
        # click on department
        department.click()
        
        time.sleep(3)
        
        # Get all elements of province
        prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "-Seleccione-" :
                
                # click on province
                province.click()
                
                time.sleep(3)
                
                # Get all elements from district
                dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                num_dist_options = len( dist.options )
                
                for dist_idx in range( num_dist_options ):
                    
                    # Get again all districts since HTML is refreshing
                    # all elements
                    dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                    district = dist.options[ dist_idx ] 
                    
                    # Get district name
                    dist_name = district.text
                    
                    if dist_name == "Inscrito" :
                        
                        # click on district
                        district.click()
                        
                        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[3]/input').click()
                        
                        time.sleep(3)
                        
                        ## Get table of presidential votes
                        # Get html at this point
                        table_path_2 = driver.find_element_by_xpath( "/html/body/div[1]/div[3]" )
                        
                        table_html_2 = table_path_2.get_attribute( 'innerHTML' )
                        table_2 = pd.read_html( table_html_2 )
                            
                        table_clean_2 = table_2[0].iloc[ 0:, [0,1,2]].copy()

                        table_clean_2[ 'department' ] = dpt_name
                        table_clean_2[ 'province' ]   = prov_name
                        table_clean_2[ 'district' ]   = dist_name
                        
                        all_tables_2[(province, district)] = table_clean_2
                        

In [12]:
final_data_2 = pd.concat( all_tables_2.values() ).reset_index( drop = True )

In [13]:
final_data_2.to_excel( 'movimientos_regional_inscrito.xlsx' , index = False )

In [ ]:
drive.quit()

#### - En proceso de inscripción

In [14]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://aplicaciones007.jne.gob.pe/srop_publico/Consulta/OrganizacionPolitica'
driver.get( url )



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\soyma\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [15]:
# Store all_tables
all_tables_3 = {}

In [16]:
# we can loop over all departments
# for dpt_idx in range( num_prov_options ):
# but it will take too much time
# We are going to do it over two departments
for dpt_idx in range( 2 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    dpt = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[1]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    # We select a different department name
    if dpt_name == "Movimiento Regional" :
        
        # click on department
        department.click()
        
        time.sleep(3)
        
        # Get all elements of province
        prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "-Seleccione-" :
                
                # click on province
                province.click()
                
                time.sleep(3)
                
                # Get all elements from district
                dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                num_dist_options = len( dist.options )
                
                for dist_idx in range( num_dist_options ):
                    
                    # Get again all districts since HTML is refreshing
                    # all elements
                    dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                    district = dist.options[ dist_idx ] 
                    
                    # Get district name
                    dist_name = district.text
                    
                    if dist_name == "En proceso de inscripción" :
                        
                        # click on district
                        district.click()
                        
                        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[3]/input').click()
                        
                        time.sleep(3)
                        
                        ## Get table of presidential votes
                        # Get html at this point
                        try:
                            
                            table_path_3 = driver.find_element_by_xpath( "/html/body/div[1]/div[3]" )

                            table_html_3 = table_path_3.get_attribute( 'innerHTML' )
                            table_3 = pd.read_html( table_html_3 )

                            table_clean_3 = table_3[0].iloc[ 0:, [0,1,2]].copy()

                            table_clean_3[ 'department' ] = dpt_name
                            table_clean_3[ 'province' ]   = prov_name
                            table_clean_3[ 'district' ]   = dist_name

                            all_tables_3[(province, district)] = table_clean_3
                        
                        except:
                            print ("no attribute 'a'")
                            continue
                        

no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'


In [17]:
final_data_3 = pd.concat( all_tables_3.values() ).reset_index( drop = True )

In [18]:
final_data_3.to_excel( 'movimientos_regional_en_proceso.xlsx' , index = False )

In [19]:
driver.quit()

### Organización Local (Provincial)

In [ ]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://aplicaciones007.jne.gob.pe/srop_publico/Consulta/OrganizacionPolitica'
driver.get( url )

In [86]:
# Store all_tables
all_tables_4 = {}

In [84]:
# we can loop over all departments
# for dpt_idx in range( num_prov_options ):
# but it will take too much time
# We are going to do it over two departments
for tp_idx in range( 3 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    tp = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[1]/select" ) )
    tipo = tp.options[ tp_idx ]
    
    # Get departmant name
    tp_name = tipo.text
    
    # We select a different department name
    if tp_name == "Organización Local (Provincial)" :
        
        # click on department
        tipo.click()
        
        time.sleep(3)
        
        # Get all elements of province
        dep = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
        num_dep_options = len( dep.options )
        
        for dep_idx in range( num_dep_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            dep = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[1]/select" ) )
            departament = dep.options[ dep_idx ]
                
            # Get province name
            dep_name = departament.text
            
            if dep_name != "-Seleccione-" :
                
                # click on province
                departament.click()
                
                time.sleep(3)
                
                prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[2]/select" ) )
                num_prov_options = len( prov.options )
                
                for prov_idx in range( num_prov_options ):
                    
                    # Get again all districts since HTML is refreshing
                    # all elements
                    prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[2]/div/div[2]/select" ) )
                    province = prov.options[ prov_idx ] 
                    
                    # Get district name
                    prov_name = province.text
                    
                    if prov_name != "-Seleccione-" :
                        
                        # click on district
                        province.click()
                                
                        # Get all elements from district
                        dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                        num_dist_options = len( dist.options )

                        for dist_idx in range( num_dist_options ):

                            # Get again all districts since HTML is refreshing
                            # all elements
                            dist = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
                            district = dist.options[ dist_idx ] 

                            # Get district name
                            dist_name = district.text

                            if dist_name == "Inscrito" :

                                # click on district
                                district.click()

                                driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[3]/input').click()

                                time.sleep(3)

                                ## Get table of presidential votes
                                # Get html at this point
                                try:

                                    table_path_4 = driver.find_element_by_xpath( "/html/body/div[1]/div[3]" )

                                    table_html_4 = table_path_4.get_attribute( 'innerHTML' )
                                    table_4 = pd.read_html( table_html_4 )

                                    table_clean_4 = table_4[0].iloc[ 0:, [0,1,2]].copy()

                                    table_clean_4[ 'tipo' ] = tp_name
                                    table_clean_4[ 'departament' ]   = dep_name
                                    table_clean_4[ 'province' ]   = prov_name
                                    table_clean_4[ 'district' ]   = dist_name

                                    all_tables_4[(departament, province, district)] = table_clean_4

                                except:
                                    print ("no attribute 'a'")
                                    continue


no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute 'a'
no attribute '

In [88]:
table_clean_4

NameError: name 'table_clean_4' is not defined

### Organización Local (Distrital)